In [1]:
import gradio as gr
import yaml
import os

# Get the current working directory
current_dir = os.getcwd()

# Construct the full path to the YAML file
yaml_file_path = os.path.join(current_dir, 'sample.yaml')

# Load the data from the YAML file
with open(yaml_file_path, 'r') as file:
    data = yaml.safe_load(file)

In [2]:
import gradio as gr
import yaml

def create_section_interface(section_name, section_data):
    with gr.Accordion(section_name.capitalize(), open=False):
        if isinstance(section_data, list):
            dynamic_fields = gr.State(value=section_data)
            def render_fields(fields):
                for item in fields:
                    with gr.Group():  # Grouping related fields together
                        if isinstance(item, dict):
                            for key, value in item.items():
                                if isinstance(value, dict):  # Handle nested dictionaries
                                    with gr.Accordion(f"{key.capitalize()}", open=False):
                                        sub_dynamic_fields = value.items()
                                        for sub_key, sub_value in sub_dynamic_fields:
                                            gr.Textbox(label=sub_key.capitalize(), placeholder=sub_value, interactive=True)

                                        # Add new field input and button within nested Accordion
                                        with gr.Row():
                                            new_sub_field_name = gr.Textbox(label="New Field Name")
                                            add_sub_button = gr.Button(value="Add Field")

                                        def add_new_sub_field(fields, name):
                                            if name:
                                                fields.append((name, ""))  # Add new field with empty placeholder
                                            return fields

                                        add_sub_button.click(fn=add_new_sub_field, inputs=[dynamic_fields, new_sub_field_name], outputs=[dynamic_fields])
                                else:
                                    gr.Textbox(label=key.capitalize(), placeholder=value, interactive=True)
                        else:
                            gr.Textbox(placeholder=item, interactive=True)

                # Input for new field name within each main Accordion
                with gr.Row():
                    new_field_name = gr.Textbox(label="New Field Name")
                    add_button = gr.Button(value="Add Field")

                # Add new field on button click
                def add_new_field(fields, name):
                    if name:
                        if isinstance(fields[0], dict):
                            fields.append({name: ""})
                        else:
                            fields.append((name, ""))  # Add new field with empty placeholder
                    return fields

                add_button.click(fn=add_new_field, inputs=[dynamic_fields, new_field_name], outputs=[dynamic_fields])

            render_fields(dynamic_fields.value)

        else:
            dynamic_fields = gr.State(value=[(key, value) for key, value in section_data.items()])
            def render_fields(fields):
                for name, hint in fields:
                    if isinstance(hint, dict):  # Handle nested dictionaries in non-list sections
                        with gr.Accordion(f"{name.capitalize()}", open=False):
                            sub_dynamic_fields = hint.items()
                            for sub_key, sub_value in sub_dynamic_fields:
                                gr.Textbox(label=sub_key.capitalize(), placeholder=sub_value, interactive=True)

                            # Add new field input and button within nested Accordion
                            with gr.Row():
                                new_sub_field_name = gr.Textbox(label="New Field Name")
                                add_sub_button = gr.Button(value="Add Field")

                            def add_new_sub_field(sub_fields, name):
                                if name:
                                    sub_fields.append((name, ""))  # Add new field with empty placeholder
                                return sub_fields

                            add_sub_button.click(fn=add_new_sub_field, inputs=[dynamic_fields, new_sub_field_name], outputs=[dynamic_fields])
                    elif name == "gender":
                        gr.Dropdown(label=name.capitalize(), choices=["Male", "Female", "Other"], value=hint, interactive=True)
                    else:
                        gr.Textbox(label=name.capitalize(), placeholder=hint, interactive=True)

                # Input for new field name within each main Accordion
                with gr.Row():
                    new_field_name = gr.Textbox(label="New Field Name")
                    add_button = gr.Button(value="Add Field")

                # Add new field on button click
                def add_new_field(fields, name):
                    if name:
                        fields.append((name, ""))  # Add new field with empty placeholder
                    return fields

                add_button.click(fn=add_new_field, inputs=[dynamic_fields, new_field_name], outputs=[dynamic_fields])

            render_fields(dynamic_fields.value)


In [3]:
with gr.Blocks() as demo:
    gr.Markdown("# User Profile")

    # Create expandable sections for each major section in the data
    for section_name, section_data in data.items():
        create_section_interface(section_name, section_data)

demo.launch(server_name="0.0.0.0", server_port=7862, inline=True, prevent_thread_lock=True)

Running on local URL:  http://0.0.0.0:7862

To create a public link, set `share=True` in `launch()`.
